### C3D+UCF_modelwithloss_training
Author : @leopauly | cnlp@leeds.ac.uk <br>
Description : Training the C3D model with new loss using UCF 101 action recognition dataset

In [1]:
## Imports
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from keras.layers.core import Dense, Dropout, Flatten
from keras.models import Sequential
import random
from keras.layers import Input
import numpy as np
from PIL import Image
from os import listdir
from scipy.ndimage import imread
import keras
from keras.models import Model
import keras.backend as K

# Custom scripts
import lscript as lsp
import modelling as md
#from DataSet.DataSet import DataSet
import dataset as dset
import ucf101_dataset as ucf

Using TensorFlow backend.


In [2]:
height=112 
width=112 
channel=3
#nb_class=3
#cluster_length=16
next_batch_start=0
batch_size=100
epochs=10
nb_classes=101
iterations=2
num_epoch=2
#strides=1
total_frames_in_seq=16
#time_step=16
#num_clusters= int( (total_frames_in_seq-cluster_length) / strides) + 1
feature_size=8192 #8192   #16384  #487

#### Getting the model

In [3]:
model = md.C3D_ucf101_training_model()

/home/ufaserv1_f/cnlp/Seeing_to_Learn/Observation-Learning/Stage_5_New_loss_function /modelling.py:334: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", name="conv1", input_shape=(16, 112, ..., padding="same")`
  input_shape=input_shape))
/home/ufaserv1_f/cnlp/Seeing_to_Learn/Observation-Learning/Stage_5_New_loss_function /modelling.py:336: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
  border_mode='valid', name='pool1'))
/home/ufaserv1_f/cnlp/Seeing_to_Learn/Observation-Learning/Stage_5_New_loss_function /modelling.py:339: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), activation="relu", name="conv2", padding="same")`
  border_mode='same', name='conv2'))
/home/ufaserv1_f/cnlp/Seeing_to_Learn/Observation-Learning/Stage_5_New_loss_function /modelling.py:341: UserWarning: Update your `MaxPooling3D

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 4, 14, 14, 256)    0         
__________

/home/ufaserv1_f/cnlp/Seeing_to_Learn/Observation-Learning/Stage_5_New_loss_function /modelling.py:363: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool5", padding="valid")`
  border_mode='valid', name='pool5'))


In [4]:
def model_lower():    
    model = Sequential()
    #if backend == 'tf':
    input_shape=(total_frames_in_seq, height, width, channel) # l, h, w, c
    #else:
    #    input_shape=(channel, total_frames_in_seq, height, width) # c, l, h, w
    model.add(Convolution3D(64, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv1',
                            input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                           border_mode='valid', name='pool1'))
    # 2nd layer group
    model.add(Convolution3D(128, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv2'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool2'))
    # 3rd layer group
    model.add(Convolution3D(256, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv3a'))
    model.add(Convolution3D(256, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv3b'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool3'))
    # 4th layer group
    model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv4a'))
    model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv4b'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool4'))
    # 5th layer group
    model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv5a'))
    model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv5b'))
    model.add(ZeroPadding3D(padding=((0, 0), (0, 1), (0, 1)), name='zeropad5'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool5'))
    model.add(Flatten())
    
    return model

def model_upper():
    model = Sequential()
    # FC layers group
    model.add(Dense(4096, activation='relu', name='fc6',input_shape=(feature_size,)))
    model.add(Dropout(.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(.5))
    model.add(Dense(101, activation='softmax', name='fc8'))
    return model
    
    
video_input = Input (shape=(total_frames_in_seq, height, width, channel))
model_bottom=model_lower()
activity_features= model_bottom(video_input)
model_top=model_upper()
class_output=model_top(activity_features)
model=Model(video_input,class_output)
print(model.summary())

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", name="conv1", input_shape=(16, 112, ..., padding="same")`
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), activation="relu", name="conv2", padding="same")`
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid")`
  app.launch_new_instance()
/opt/conda/lib/python3.6/site

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 112, 112, 3)   0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 8192)              27655936  
_________________________________________________________________
sequential_3 (Sequential)    (None, 101)               50753637  
Total params: 78,409,573
Trainable params: 78,409,573
Non-trainable params: 0
_________________________________________________________________
None


#### Compiling model and defining loss function 

In [5]:
train_images, train_labels_uncat, next_batch_start, _ = ucf. read_vid_and_label('UCF101_data_preparation/train.list',batch_size,-1,16,112,normalisation=True)
train_labels=keras.utils.to_categorical(train_labels_uncat, num_classes=nb_classes)
print(train_labels)

Loading /nobackup/leopauly/UCF101/UCF-101/BreastStroke/v_BreastStroke_g03_c01...
Loading /nobackup/leopauly/UCF101/UCF-101/Skijet/v_Skijet_g11_c01...
Loading /nobackup/leopauly/UCF101/UCF-101/HeadMassage/v_HeadMassage_g03_c04...
Loading /nobackup/leopauly/UCF101/UCF-101/CricketShot/v_CricketShot_g23_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/FrisbeeCatch/v_FrisbeeCatch_g02_c01...
Loading /nobackup/leopauly/UCF101/UCF-101/JumpingJack/v_JumpingJack_g19_c05...
Loading /nobackup/leopauly/UCF101/UCF-101/SkateBoarding/v_SkateBoarding_g10_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/HulaHoop/v_HulaHoop_g20_c02...
Loading /nobackup/leopauly/UCF101/UCF-101/PlayingDaf/v_PlayingDaf_g18_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/Rowing/v_Rowing_g20_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/HorseRiding/v_HorseRiding_g15_c01...
Loading /nobackup/leopauly/UCF101/UCF-101/UnevenBars/v_UnevenBars_g06_c04...
Loading /nobackup/leopauly/UCF101/UCF-101/BoxingSpeedBag/v_BoxingSpeedBag_

Loading /nobackup/leopauly/UCF101/UCF-101/Typing/v_Typing_g05_c01...
Loading /nobackup/leopauly/UCF101/UCF-101/HighJump/v_HighJump_g20_c05...
Loading /nobackup/leopauly/UCF101/UCF-101/WritingOnBoard/v_WritingOnBoard_g03_c05...
Loading /nobackup/leopauly/UCF101/UCF-101/Kayaking/v_Kayaking_g08_c06...
Loading /nobackup/leopauly/UCF101/UCF-101/StillRings/v_StillRings_g08_c04...
Loading /nobackup/leopauly/UCF101/UCF-101/SoccerJuggling/v_SoccerJuggling_g15_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/IceDancing/v_IceDancing_g09_c01...
Loading /nobackup/leopauly/UCF101/UCF-101/SalsaSpin/v_SalsaSpin_g17_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/CuttingInKitchen/v_CuttingInKitchen_g12_c01...
Loading /nobackup/leopauly/UCF101/UCF-101/WritingOnBoard/v_WritingOnBoard_g23_c01...
Loading /nobackup/leopauly/UCF101/UCF-101/MoppingFloor/v_MoppingFloor_g25_c02...
Loading /nobackup/leopauly/UCF101/UCF-101/BoxingPunchingBag/v_BoxingPunchingBag_g04_c04...
(100, 16, 112, 112, 3)
[[ 0.  0.  0. ...

In [21]:
def new_loss(y_true, y_pred):
    print('entered loss function')
    loss_1=K.categorical_crossentropy(y_true, y_pred)

    #y_true=train_labels
    y_true_classes=K.argmax(y_true,axis=1)
    k=np.unique(y_true_classes)
    indexes=np.ones([len(k),batch_size],dtype=np.int8)*(nb_classes+1)
    for w in range(len(k)):
        t=0
        for i in range(batch_size):
            if y_true_classes[i] == k[w]:
                indexes[w][t]=int(i)
                t=t+1
    loss_2=0
    for i in range(len(k)):
        w=[x for x in indexes[i][:] if x!=(nb_classes+1)]
        loss_2=loss_2+K.sum(K.var(activity_features[w],axis=0))
        #print(K.eval(loss_2))

    return loss_1+loss_2

In [22]:
model.compile(optimizer='adam',loss=new_loss,metrics=['accuracy'])

entered loss function


#### Training 

In [23]:
for i in range(iterations):
    train_images, train_labels_uncat, next_batch_start, _ = ucf. read_vid_and_label('UCF101_data_preparation/train.list',10,-1,16,112,normalisation=True)
    train_labels=keras.utils.to_categorical(train_labels_uncat, num_classes=nb_classes)
    #print(train_images.shape)
    #print(train_labels_uncat)
    model.fit(train_images, train_labels,batch_size=batch_size, epochs=num_epoch, verbose=1,validation_split=0.0, validation_data=None, shuffle=False)
    #print(next_batch_start)

Loading /nobackup/leopauly/UCF101/UCF-101/Archery/v_Archery_g08_c04...
Loading /nobackup/leopauly/UCF101/UCF-101/JumpingJack/v_JumpingJack_g14_c04...
Loading /nobackup/leopauly/UCF101/UCF-101/BaseballPitch/v_BaseballPitch_g09_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/Archery/v_Archery_g02_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/Knitting/v_Knitting_g04_c04...
Loading /nobackup/leopauly/UCF101/UCF-101/HammerThrow/v_HammerThrow_g03_c05...
Loading /nobackup/leopauly/UCF101/UCF-101/Knitting/v_Knitting_g22_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/TennisSwing/v_TennisSwing_g03_c02...
Loading /nobackup/leopauly/UCF101/UCF-101/FrisbeeCatch/v_FrisbeeCatch_g09_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/FrontCrawl/v_FrontCrawl_g06_c03...
Loading /nobackup/leopauly/UCF101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01...
Loading /nobackup/leopauly/UCF101/UCF-101/SalsaSpin/v_SalsaSpin_g12_c05...
Loading /nobackup/leopauly/UCF101/UCF-101/BlowDryHair/v_BlowDryHair_g23_c0

In [ ]:
test_images, test_labels_uncat, next_batch_start, _ = ucf. read_vid_and_label('UCF101_data_preparation/test.list',1000,-1,16,112,normalisation=True)
test_labels=keras.utils.to_categorical(test_labels_uncat, num_classes=nb_classes)
#print(test_images.shape)
print(test_labels_uncat)
score = model.evaluate(test_images, test_labels, batch_size=50)
print(score)

